<a href="https://colab.research.google.com/github/willcoomans5/mlb_predictions/blob/main/mlb_game_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install torch pybaseball scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [11]:
import os
import sys
import pandas as pd
import numpy as np
import pybaseball as pyb
import torch
from datetime import date
from sklearn.ensemble import RandomForestClassifier
import joblib


# pyb.cache.enable()
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)


In [6]:
path = '/content/drive/MyDrive/mlbpredictions/'
fname = path + 'gl2020_23/gl2023.txt'
df = pd.read_csv(fname, header=None)
df.shape

(2430, 161)

In [7]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',
           'misc_info','acqui_info'
           ]

df.columns = colnames
df

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,...,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
0,20230330,0,Thu,MIL,NL,1,CHN,NL,1,0,4,51,D,NaN,NaN,NaN,CHI11,36054.0,141,000000000,00400000x,29,4,0,0,0,0,0,0,0,5,0,12,0,0,2,0,7,4,4,4,0,0,24,12,1,0,1,0,30,...,adamw002,Willy Adames,6,tellr001,Rowdy Tellez,3,contw002,William Contreras,2,urial001,Luis Urias,5,mitcg001,Garrett Mitchell,8,andeb006,Brian Anderson,9,turab002,Brice Turang,4,hoern001,Nico Hoerner,4,swand001,Dansby Swanson,6,happi001,Ian Happ,7,bellc002,Cody Bellinger,8,manct001,Trey Mancini,10,gomey001,Yan Gomes,2,hosme001,Eric Hosmer,3,wisdp001,Patrick Wisdom,5,mastm001,Miles Mastrobuoni,9,NaN,Y
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,4,54,D,NaN,NaN,NaN,CIN09,44063.0,182,001300010,100120000,30,6,1,0,1,4,1,1,0,9,0,11,2,0,1,0,9,5,4,4,0,0,27,9,1,0,2,0,33,...,mccua001,Andrew McCutchen,10,santc002,Carlos Santana,3,smitc008,Canaan Smith-Njigba,9,hayek001,Ke'Bryan Hayes,5,suwij001,Jack Suwinski,8,bae-j001,Ji Hwan Bae,4,hedga001,Austin Hedges,2,indij001,Jonathan India,4,friet001,TJ Friedl,8,fralj001,Jake Fraley,10,stept001,Tyler Stephenson,2,voslj001,Jason Vosler,3,myerw001,Wil Myers,9,stees001,Spencer Steer,5,bensw001,Will Benson,7,garcj007,Jose Garcia,6,NaN,Y
2,20230330,0,Thu,ARI,NL,1,LAN,NL,1,2,8,51,N,NaN,NaN,NaN,LOS03,52075.0,155,110000000,00203201x,28,4,1,0,0,2,0,1,1,0,0,8,0,0,2,0,1,5,8,8,0,0,24,7,1,0,0,0,34,...,gurrl001,Lourdes Gurriel,7,walkc002,Christian Walker,3,longe001,Evan Longoria,5,ahmen001,Nick Ahmed,6,carrc005,Corbin Carroll,8,moreg001,Gabriel Moreno,2,mccaj003,Jake McCarthy,9,bettm001,Mookie Betts,9,freef001,Freddie Freeman,3,smitw003,Will Smith,2,muncm001,Max Muncy,5,martj006,J.D. Martinez,10,perad001,David Peralta,7,vargm001,Miguel Vargas,4,outmj002,James Outman,8,rojam002,Miguel Rojas,6,NaN,Y
3,20230330,0,Thu,NYN,NL,1,MIA,NL,1,5,3,54,D,NaN,NaN,NaN,MIA02,31397.0,162,001002200,000003000,32,8,1,0,0,5,0,2,1,5,0,5,1,0,0,0,8,4,3,3,0,0,27,8,0,0,2,0,30,...,lindf001,Francisco Lindor,6,alonp001,Pete Alonso,3,mcnej002,Jeff McNeil,4,canhm001,Mark Canha,7,voged001,Daniel Vogelbach,10,escoe001,Eduardo Escobar,5,narvo001,Omar Narvaez,2,arral001,Luis Arraez,4,seguj002,Jean Segura,5,coopg002,Garrett Cooper,3,chisj001,Jazz Chisholm,8,solej001,Jorge Soler,10,garca003,Avisail Garcia,9,delab001,Bryan De La Cruz,7,stalj001,Jacob Stallings,2,wendj002,Joey Wendle,6,NaN,Y
4,20230330,0,Thu,COL,NL,1,SDN,NL,1,7,2,54,N,NaN,NaN,NaN,SAN02,45103.0,176,100031200,100100000,44,17,4,0,3,7,0,0,0,1,0,17,1,0,0,0,11,4,2,2,0,1,27,9,3,0,2,0,32,...,blacc001,Charlie Blackmon,10,cronc002,C.J. Cron,3,monte001,Elehuris Montero,5,mcmar001,Ryan McMahon,4,diaze005,Elias Diaz,2,casth001,Harold Castro,7,tovae001,Ezequiel Tovar,6,grist001,Trent Grisham,8,sotoj001,Juan Soto,7,machm001,Manny Machado,5,bogax001,Xander Bogaerts,6,cronj001,Jake Cronenworth,3,carpm002,Matt Carpenter,10,nolaa002,Austin Nola,2,kim-h002,Ha-Seong Kim,4,dahld001,David Dahl,9,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [8]:
teams = np.array([df.groupby('team_v').agg('first').index])
teams

array([['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CHA', 'CHN', 'CIN', 'CLE',
        'COL', 'DET', 'HOU', 'KCA', 'LAN', 'MIA', 'MIL', 'MIN', 'NYA',
        'NYN', 'OAK', 'PHI', 'PIT', 'SDN', 'SEA', 'SFN', 'SLN', 'TBA',
        'TEX', 'TOR', 'WAS']], dtype=object)

In [9]:
statcast_data = pyb.statcast(start_dt='2023-03-30', end_dt='2023-11-01')

This is a large query, it may take a moment to complete


/usr/local/lib/python3.10/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 217/217 [05:52<00:00,  1.62s/it]


In [13]:
print(statcast_data.shape)
statcast_data.to_csv(path + "statcast_2023.csv")
statcast_data.head()

(729774, 94)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
120,CU,2023-11-01,84.9,-1.19,6.12,"Sborz, Josh",606466,622250,strikeout,called_strike,<NA>,<NA>,<NA>,<NA>,1,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,2,NaN,2,2,2023,1.14,-0.9,-0.54,3.55,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,641680,<NA>,<NA>,-0.726868,-123.595406,2.114037,11.719058,27.037078,-42.00668,3.52,1.65,<NA>,<NA>,<NA>,83.8,2515,5.8,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.73,<NA>,0.0,0.0,1,0,0,<NA>,77,5,Curveball,0,5,0,5,5,0,0,5,Infield shade,Standard,26,0.0,-0.09,<NA>,<NA>
125,FF,2023-11-01,96.6,-0.69,6.24,"Sborz, Josh",606466,622250,NaN,ball,<NA>,<NA>,<NA>,<NA>,14,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,B,<NA>,NaN,1,2,2023,-0.35,1.45,0.87,1.76,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,641680,<NA>,<NA>,4.9136,-140.261187,-9.567087,-5.773926,32.716582,-11.02041,3.65,1.62,<NA>,<NA>,<NA>,95.9,2442,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.44,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,77,4,4-Seam Fastball,0,5,0,5,5,0,0,5,Strategic,Standard,206,0.0,0.016,<NA>,<NA>
128,CU,2023-11-01,84.5,-1.27,6.11,"Sborz, Josh",606466,622250,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,1,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,<NA>,NaN,1,1,2023,1.06,-0.87,-0.58,3.32,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,641680,<NA>,<NA>,-0.481056,-123.00404,1.607595,10.75001,26.459795,-41.523154,3.65,1.65,<NA>,<NA>,<NA>,83.5,2339,5.8,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.72,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,77,3,Curveball,0,5,0,5,5,0,0,5,Standard,Standard,24,0.0,-0.027,<NA>,<NA>
134,FF,2023-11-01,95.4,-0.8,6.23,"Sborz, Josh",606466,622250,NaN,foul,<NA>,<NA>,<NA>,<NA>,2,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,<NA>,NaN,1,0,2023,-0.63,1.56,0.16,3.34,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,641680,<NA>,<NA>,3.942539,-138.762249,-5.389822,-9.112213,34.274743,-11.122984,3.62,1.72,<NA>,<NA>,<NA>,94.7,2392,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.39,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,77,2,4-Seam Fastball,0,5,0,5,5,0,0,5,Standard,Standard,204,0.0,-0.022,<NA>,<NA>
136,FF,2023-11-01,95.6,-0.83,6.19,"Sborz, Josh",606466,622250,NaN,ball,<NA>,<NA>,<NA>,<NA>,14,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,B,<NA>,NaN,0,0,2023,-0.14,1.67,0.42,1.6,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,641680,<NA>,<NA>,3.59041,-138.889526,-10.126828,-2.678258,33.103774,-8.516644,3.68,1.75,<NA>,<NA>,<NA>,94.8,2289,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.45,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,77,1,4-Seam Fastball,0,5,0,5,5,0,0,5,Standard,Standard,207,0.0,0.016,<NA>,<NA>


In [19]:
# use release_spin_rate, spin_axis, vx0, vy0, vz0, pfx_x, pfx_z
# maybe release_extension, release_pos_x, release_pos_z

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length


In [15]:
sc = pd.read_csv(path + "statcast_2023.csv")
sc

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,120,CU,2023-11-01,84.9,-1.19,6.12,"Sborz, Josh",606466,622250,strikeout,called_strike,NaN,NaN,NaN,NaN,1.0,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,2.0,NaN,2,2,2023,1.14,-0.90,-0.54,3.55,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,641680,NaN,NaN,-0.726868,-123.595406,2.114037,11.719058,27.037078,-42.006680,3.52,1.65,NaN,NaN,NaN,83.8,2515.0,5.8,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.73,NaN,0.0,0.0,1.0,0.0,0.0,NaN,77,5,Curveball,0,5,0,5,5,0,0,5,Infield shade,Standard,26.0,0.0,-0.090,NaN,NaN
1,125,FF,2023-11-01,96.6,-0.69,6.24,"Sborz, Josh",606466,622250,NaN,ball,NaN,NaN,NaN,NaN,14.0,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,B,NaN,NaN,1,2,2023,-0.35,1.45,0.87,1.76,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,641680,NaN,NaN,4.913600,-140.261187,-9.567087,-5.773926,32.716582,-11.020410,3.65,1.62,NaN,NaN,NaN,95.9,2442.0,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,4,4-Seam Fastball,0,5,0,5,5,0,0,5,Strategic,Standard,206.0,0.0,0.016,NaN,NaN
2,128,CU,2023-11-01,84.5,-1.27,6.11,"Sborz, Josh",606466,622250,NaN,called_strike,NaN,NaN,NaN,NaN,1.0,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,NaN,NaN,1,1,2023,1.06,-0.87,-0.58,3.32,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,641680,NaN,NaN,-0.481056,-123.004040,1.607595,10.750010,26.459795,-41.523154,3.65,1.65,NaN,NaN,NaN,83.5,2339.0,5.8,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,3,Curveball,0,5,0,5,5,0,0,5,Standard,Standard,24.0,0.0,-0.027,NaN,NaN
3,134,FF,2023-11-01,95.4,-0.80,6.23,"Sborz, Josh",606466,622250,NaN,foul,NaN,NaN,NaN,NaN,2.0,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,S,NaN,NaN,1,0,2023,-0.63,1.56,0.16,3.34,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,641680,NaN,NaN,3.942539,-138.762249,-5.389822,-9.112213,34.274743,-11.122984,3.62,1.72,NaN,NaN,NaN,94.7,2392.0,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,2,4-Seam Fastball,0,5,0,5,5,0,0,5,Standard,Standard,204.0,0.0,-0.022,NaN,NaN
4,136,FF,2023-11-01,95.6,-0.83,6.19,"Sborz, Josh",606466,622250,NaN,ball,NaN,NaN,NaN,NaN,14.0,Ketel Marte called out on strikes.,W,L,R,AZ,TEX,B,NaN,NaN,0,0,2023,-0.14,1.67,0.42,1.60,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,641680,NaN,NaN,3.590410,-138.889526,-10.126828,-2.678258,33.103774,-8.516644,3.68,1.75,NaN,NaN,NaN,94.8,2289.0,6.1,748534,622250,641680,663993,543760,673962,608369,694497,665750,608671,54.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,1,4-Seam Fastball,0,5,0,5,5,0,0,5,Standard,Standard,207.0,0.0,0.016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,